In [41]:
import cv2
import time
import numpy as np
import math
from comtypes import CLSCTX_ALL
from pycaw.pycaw import AudioUtilities, IAudioEndpointVolume
import HandTrackingModule as htm

In [42]:
wCam, hCam = 1280, 720

In [43]:
cap = cv2.VideoCapture(0)
cap.set(3, wCam)
cap.set(4, hCam)
pTime = 0

In [44]:
detector = htm.handDetector()

devices = AudioUtilities.GetSpeakers()
interface = devices.Activate(
    IAudioEndpointVolume._iid_, CLSCTX_ALL, None)
volume = interface.QueryInterface(IAudioEndpointVolume)
# volume.GetMute()
# volume.GetMasterVolumeLevel()
volRange = volume.GetVolumeRange()
volume.SetMasterVolumeLevel(0, None)
minVol = volRange[0]
maxVol = volRange[1]
vol = 0
volBar = 400
volPer = 0

In [45]:
while True:
    # Capture frame from video feed
    success, img = cap.read()
    img = detector.findHands(img)
    lmList = detector.findPosition(img, draw=False)
    
    if len(lmList) != 0:
        # Retrieve coordinates of index and thumb
        x1, y1 = lmList[4][1], lmList[4][2]  # Thumb tip
        x2, y2 = lmList[8][1], lmList[8][2]  # Index tip
        cx, cy = (x1 + x2) // 2, (y1 + y2) // 2  # Center point

        # Draw circles with gradient colors
        cv2.circle(img, (x1, y1), 12, (240, 128, 128), -1)
        cv2.circle(img, (x2, y2), 12, (135, 206, 250), -1)
        cv2.circle(img, (cx, cy), 10, (255, 255, 255), -1)

        # Draw a smooth line with thicker width
        cv2.line(img, (x1, y1), (x2, y2), (50, 205, 50), 4)
        
        # Calculate length between thumb and index finger
        length = math.hypot(x2 - x1, y2 - y1)

        # Map hand distance to volume range
        vol = np.interp(length, [50, 300], [minVol, maxVol])
        volPer = np.interp(length, [50, 300], [0, 100])
        volBar = np.interp(length, [50, 300], [400, 150])

        # Set volume based on distance
        volume.SetMasterVolumeLevel(vol, None)

        # Change color when distance is small
        if length < 50:
            cv2.circle(img, (cx, cy), 15, (0, 0, 0), -1)

    # Draw a sleek, gradient-filled volume bar
    overlay = img.copy()
    cv2.rectangle(overlay, (50, 150), (85, 400), (255, 255, 255), 3)
    cv2.rectangle(overlay, (50, int(volBar)), (85, 400), (0, 255, 0), -1)
    cv2.addWeighted(overlay, 0.6, img, 0.4, 0, img)

    # Display volume percentage with a transparent background for the text
    cv2.rectangle(img, (40, 680), (200, 720), (0, 0, 0, 150), -1)
    cv2.putText(img, f'Volume: {int(volPer)}%', (45, 710), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)

    # Calculate and display FPS with a transparent background
    cTime = time.time()
    fps = 1 / (cTime - pTime)
    pTime = cTime
    cv2.rectangle(img, (30, 30), (150, 70), (0, 0, 0, 150), -1)
    cv2.putText(img, f'FPS: {int(fps)}', (40, 60), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)

    # Show image frame
    cv2.imshow('Enhanced Hand Tracking Volume Control', img)
    cv2.waitKey(1)

KeyboardInterrupt: 